In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
os.listdir('./')

['.ipynb_checkpoints',
 '0206.csv',
 '0207.csv',
 '0207_result.csv',
 '0208.csv',
 '0208_result.csv',
 '0210.csv',
 '0211.csv',
 '0212.csv',
 '0213.csv',
 '0214.csv',
 '3차',
 'call_data.csv',
 'call_model.pt',
 'f_call_data.csv',
 'f_mu_data.csv',
 'model 검증.ipynb',
 'model 검증.xlsx',
 'mu_data.csv',
 'mu_model.pt',
 '필터링모델_검증(0203) .ipynb',
 '필터링모델_검증(0206).ipynb',
 '필터링모델_검증(0208).ipynb']

In [4]:
def get_predict_data(text): 
    request = {'Time':[], 'Request':[]}
    response = {'Time':[], 'Response':[]}

    cnt = 0
    with open(f"{text}", 'r') as f:
        data = f.readlines()
        for line in data:
            if cnt%2==0:
                request['Time'].append(line[7:26])
                request['Request'].append(line[42:])
            else:
                response['Time'].append(line[7:26])
                response['Response'].append(line[42:])                        
            cnt+=1
            
    
    req = pd.DataFrame(request)
    res = pd.DataFrame(response)

    print(req.shape[0])
    print(res.shape[0])

        
    return req,res

In [5]:
from ast import literal_eval

def get_request_df(req):
    req['len'] = req['Request'].apply(lambda x: len(x.split(", {")))
    req['cleansing_Req'] = req['Request'].apply(lambda x: x.replace("}","").replace("\n","").replace("]","").split(", {"))
    req['cleansing_Time'] = (req['Time']+',') *req['len']
    req['cleansing_Time'] = req['cleansing_Time'].apply(lambda x:x.split(','))

    request_dict = {'Time' : [], 'Request': []}

    for idx in range(len(req)):
        for t, r in zip(req.loc[idx,'cleansing_Time'], req.loc[idx,'cleansing_Req']):
            request_dict['Time'].append(t)
            request_dict['Request'].append(r)

    req_df= pd.DataFrame(request_dict)
    req_df['Request'] = req_df['Request'].apply(lambda x: '{'+x+'}')
    req_df['Request'] = req_df['Request'].apply(lambda x: literal_eval(x))

    req_df['autoNo']= req_df['Request'].apply(lambda x: x['autoNo'])
    req_df['memNo']=req_df['Request'].apply(lambda x: x['memNo'])
    req_df['mateConts']=req_df['Request'].apply(lambda x: x['mateConts'])
    req_df['familyConts']=  req_df['Request'].apply(lambda x: x['familyConts'])

    return req_df

In [6]:
def get_response_df(res):
    res['len'] = res['Response'].apply(lambda x: len(x.split(", {")))
    res['cleansing_Req'] = res['Response'].apply(lambda x: x.replace("}","").replace("\n","").split(", {"))
    res['cleansing_Time'] = (res['Time']+',') *res['len']
    res['cleansing_Time'] = res['cleansing_Time'].apply(lambda x:x.split(','))

    response_dict = {'Time' : [], 'Response': []}

    for idx in range(len(res)):
        for t, r in zip(res.loc[idx,'cleansing_Time'], res.loc[idx,'cleansing_Req']):
            response_dict['Time'].append(t)
            response_dict['Response'].append(r)

    res_df = pd.DataFrame(response_dict)
    res_df['Response'] = res_df['Response'].apply(lambda x: '{'+x+'}')
    res_df['Response'] = res_df['Response'].apply(lambda x: x.replace(']}','}'))
    res_df['Response'] = res_df['Response'].apply(lambda x: literal_eval(x))

    res_df['autoNo']= res_df['Response'].apply(lambda x: x['autoNo'])
    res_df['memNo']= res_df['Response'].apply(lambda x: x['memNo'])
    res_df['matePred']= res_df['Response'].apply(lambda x: x['matePred'])
    res_df['familyPred']= res_df['Response'].apply(lambda x: x['familyPred'])

    res_df['mate_mu'] = res_df['matePred'].apply(lambda x: x[0])
    res_df['mate_call'] = res_df['matePred'].apply(lambda x: x[1])
    res_df['family_mu'] = res_df['familyPred'].apply(lambda x: x[0])
    res_df['family_call'] = res_df['familyPred'].apply(lambda x: x[1])
    
    return res_df

In [7]:
def get_final_df(req, res):
    
    req = req.drop(columns=['Time','Request','autoNo']).reset_index(drop=True)
    res = res.drop(columns=['Time','Response','autoNo','matePred','familyPred']).reset_index(drop=True)
    final = pd.merge(req,res, how='inner').drop_duplicates(['memNo','mateConts']).reset_index(drop=True)
    
    return final

In [8]:
def get_except_df(df,final):
    
    mem_lst = final['memNo'].tolist()
    df = df.loc[~df['memNo'].isin(mem_lst)]
    
    return df

## `15일`

In [10]:
req15, res15 = get_predict_data('ai_conts_electra_main.log.2023-02-15')

req_df15= get_request_df(req15)
res_df15= get_response_df(res15)

display(req_df15.head(3))
display(res_df15.head(3))

final15 = get_final_df(req_df15, res_df15)
final15

final15.to_csv('0215.csv',index=False, encoding='utf-8-sig')

605
605


,Time,Request,autoNo,memNo,mateConts,familyConts
0,2023-02-15 00:01:00,"{'autoNo': 59920, 'memNo': 1848915, 'mateConts...",59920,1848915,안녕하세요~ 저는 중소법인을 운영중인 중소기업 사장입니다. 유쾌하고 친절한 편이고 ...,할머니 아버지 어머니 저 이렇게 네명이 함께 살고있고 남동생은 집 근처에서 혼자 살...
1,2023-02-15 00:01:00,"{'autoNo': 59922, 'memNo': 1734332, 'mateConts...",59922,1734332,"PR회사 연구원으로 광고홍보, 리서치, 데이터 통계분석 담당 연구원으로 활동했습니다...","아버지, 어머니, 누나, 형이 있고 현재는 독립해서 혼자 지내는 중입니다."
2,2023-02-15 00:01:00,"{'autoNo': 59924, 'memNo': 2042310, 'mateConts...",59924,2042310,반갑습니다 저는 인생에 한번 굴곡을 지닌 사람이입니다 하,저는 3남2녀중 막내이며 슬하에 딸이 2명이 있습니다 큰애는 30살이며 곧 결혼을 ...


,Time,Response,autoNo,memNo,matePred,familyPred,mate_mu,mate_call,family_mu,family_call
0,2023-02-15 00:01:02,"{'autoNo': 59920, 'memNo': 1848915, 'matePred'...",59920,1848915,"[0, 0]","[0, 0]",0,0,0,0
1,2023-02-15 00:01:02,"{'autoNo': 59922, 'memNo': 1734332, 'matePred'...",59922,1734332,"[0, 0]","[0, 0]",0,0,0,0
2,2023-02-15 00:01:02,"{'autoNo': 59924, 'memNo': 2042310, 'matePred'...",59924,2042310,"[0, 0]","[0, 0]",0,0,0,0
